# 三轴示教
“具身稚能”[小米微电机Cybergear上手开发](https://space.bilibili.com/1969563580/channel/collectiondetail?sid=1683510) 《[两千元打造拖动示教无刷伺服机械臂](https://www.bilibili.com/video/BV1vu411T7d4/)》 视频代码

In [ ]:
import os
import sys
# 添加pcan_cybergear库的路径
sys.path.append(os.path.join("..", "cybergear"))

from pcan_cybergear import CANMotorController
import can
import logging
import time
# Initialize logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# Connect to the CAN bus with 1 Mbit/s bitrate
bus = can.interface.Bus(bustype="pcan", channel="PCAN_USBBUS1", bitrate=1000000)
motor1 = CANMotorController(bus, motor_id=101, main_can_id=254)
motor2 = CANMotorController(bus, motor_id=102, main_can_id=254)
motor3 = CANMotorController(bus, motor_id=103, main_can_id=254)

motors = [motor1, motor2, motor3]

In [ ]:
motor1.write_param_table("limit_cur", 0.5)
motor1.write_param_table("loc_kp", 8)
motor1.write_param_table("spd_kp", 2)
motor1.write_param_table("spd_ki", 0.03)

In [ ]:
motor2.write_param_table("limit_cur", 2)
motor2.write_param_table("loc_kp", 8)
motor2.write_param_table("spd_kp", 2)
motor2.write_param_table("spd_ki", 0.03)

In [ ]:
motor3.write_param_table("limit_cur", 1)
motor3.write_param_table("loc_kp", 8)
motor3.write_param_table("spd_kp", 2)
motor3.write_param_table("spd_ki", 0.02)

## 初始化

In [ ]:
for motor in motors:
    motor.disable()
    motor.set_0_pos()
    motor.set_run_mode(motor.RunModes.POSITION_MODE) # 位置模式
    motor.write_single_param("loc_ref", value=0) # 目标位置

motor1.write_single_param("limit_spd", value=10) # 最大速度 rad/s
motor2.write_single_param("limit_spd", value=1.5) # 最大速度 rad/s
motor3.write_single_param("limit_spd", value=10) # 最大速度 rad/s

In [ ]:
for motor in motors:
    motor.enable()

In [ ]:
import time
motor2.write_single_param("loc_ref", value=-0.5)
time.sleep(1.5)
motor2.write_single_param("loc_ref", value=0.5)
time.sleep(1.5)
motor2.write_single_param("loc_ref", value=0)

In [ ]:
motor3.set_motor_position_control(limit_spd=10, loc_ref=1)

In [ ]:
for motor in motors:
    motor.disable()
    motor.set_0_pos()

# 示教

In [ ]:
import csv
import time

# 保存点位信息到CSV文件
def save_positions_to_csv(positions, filename="positions.csv"):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["motor1", "motor2", "motor3"])
        for position in positions:
            writer.writerow([position[0], position[1], position[2]])

# 从CSV文件加载点位信息
def load_positions_from_csv(filename="positions.csv"):
    positions = []
    with open(filename, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # skip header
        for row in csv_reader:
            positions.append([float(row[0]), float(row[1]), float(row[2])])
    return positions


In [ ]:
positions = []

while True:
    action = input("Press Enter to record the current position, or 'x' to exit: ")

    if action == "x":
        break
    _, pos1, _ = motor1.write_single_param("loc_ref", value=0)
    _, pos2, _ = motor2.write_single_param("loc_ref", value=0)
    _, pos3, _ = motor3.write_single_param("loc_ref", value=0)
    pos = [pos1, pos2, pos3]
    logging.info(pos)
    positions.append(pos)

In [ ]:
len(positions)

In [ ]:
save_positions_to_csv(positions)

In [ ]:
import pandas as pd
pd.read_csv("positions.csv")

In [ ]:
for motor in motors:
    motor.enable()
    motor.write_single_param("loc_ref", value=0)

In [ ]:
loaded_positions = load_positions_from_csv()

for position in loaded_positions:
    logging.info(position)
    motor1.write_single_param("loc_ref", value=position[0])
    motor2.write_single_param("loc_ref", value=position[1])
    motor3.write_single_param("loc_ref", value=position[2])

    time.sleep(1.5)  # 等待电机移动到目标位置

motor1.write_single_param("loc_ref", value=0)
motor2.write_single_param("loc_ref", value=0)
motor3.write_single_param("loc_ref", value=0)

In [ ]:
for motor in motors:
    motor.disable()